In [27]:
%%writefile matrixmul.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <fstream>
using namespace std;

__global__ void matrixMultiply(long long *A, long long *B, long long *R, int m, int n, int p) {
    int mat_id = blockIdx.z;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < m && col < p) {
        long long sum = 0;
        for(int i = 0; i < n; ++i) {
            long long a = A[mat_id * m * n + row * n + i];
            long long b = B[mat_id * n * p + i * p + col];
            sum += a * b;
        }
        R[mat_id * m * p + row * p + col] = sum;
    }
}

int main(int argc, char* argv[]) {
    if (argc != 6) {
        printf("Usage: %s <T> <K> <M> <N> <P>\n", argv[0]);
        return -1;
    }

    int T = atoi(argv[1]);
    int K = atoi(argv[2]);
    int M = atoi(argv[3]);
    int N = atoi(argv[4]);
    int P = atoi(argv[5]);

    size_t sizeA = K * M * N * sizeof(long long);
    size_t sizeB = K * N * P * sizeof(long long);
    size_t sizeR = K * M * P * sizeof(long long);

    long long *h_A = (long long*)malloc(sizeA);
    long long *h_B = (long long*)malloc(sizeB);
    long long *h_R = (long long*)malloc(sizeR);

    for(int k = 0; k < K; k++) {
        for(int i = 0; i < M; i++) {
            for(int j = 0; j < N; j++) {
                h_A[k * M * N + i * N + j] = rand() % 10;
            }
        }
        for(int i = 0; i < N; i++) {
            for(int j = 0; j < P; j++) {
                h_B[k * N * P + i * P + j] = rand() % 10;
            }
        }
    }

    long long *d_A, *d_B, *d_R;
    cudaMalloc(&d_A, sizeA);
    cudaMalloc(&d_B, sizeB);
    cudaMalloc(&d_R, sizeR);

    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(T, T);
    dim3 numBlocks((P + T - 1) / T, (M + T - 1) / T, K);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    for (int iter = 0; iter < 100; iter++) {
        matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_R, M, N, P);
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(h_R, d_R, sizeR, cudaMemcpyDeviceToHost);

    ofstream out("output.txt");

    out << "Matrix A[0] (" << M << "x" << N << "):\n";
    for(int i = 0; i < M; i++) {
        for(int j = 0; j < N; j++) {
            out << h_A[0 * M * N + i * N + j] << " ";
        }
        out << "\n";
    }

    out << "\nMatrix B[0] (" << N << "x" << P << "):\n";
    for(int i = 0; i < N; i++) {
        for(int j = 0; j < P; j++) {
            out << h_B[0 * N * P + i * P + j] << " ";
        }
        out << "\n";
    }

    out << "\n C[0] (" << M << "x" << P << "):\n";
    for(int i = 0; i < M; i++) {
        for(int j = 0; j < P; j++) {
            out << h_R[0 * M * P + i * P + j] << " ";
        }
        out << "\n";
    }

    out.close();

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_R);
    free(h_A);
    free(h_B);
    free(h_R);

    return 0;
}

Overwriting matrixmul.cu


In [28]:
!nvcc -arch=sm_75 matrixmul.cu -o matrixmul

In [29]:
!time ./matrixmul 4 100 40 40 40


real	0m0.151s
user	0m0.038s
sys	0m0.109s


In [25]:
!time ./matrixmul 2 100 40 40 40


real	0m0.178s
user	0m0.062s
sys	0m0.111s


In [26]:
!time ./matrixmul 1 100 40 40 40


real	0m0.291s
user	0m0.175s
sys	0m0.108s


In [30]:
!time ./matrixmul 1 1 2 2 2


real	0m0.123s
user	0m0.014s
sys	0m0.105s
